# Nuke button

In [ ]:
# Delete all files and folders in the working directory. Use with caution!
import shutil
shutil.rmtree("/kaggle/working", ignore_errors=True)

# Clone repo

In [2]:
%cd /kaggle/working
# !git clone https://github.com/BinhPQ2/manga_read_along.git
!git clone -b magiv2 https://github.com/BinhPQ2/manga_read_along.git

/kaggle/working
Cloning into 'manga_read_along'...
remote: Enumerating objects: 620, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 620 (delta 24), reused 65 (delta 19), pack-reused 544 (from 1)
Receiving objects: 100% (620/620), 80.20 MiB | 46.69 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [3]:
%cd /kaggle/working/manga_read_along

/kaggle/working/manga_read_along


In [4]:
!git submodule init
!git submodule update --remote

Submodule 'magi_functional' (https://github.com/BinhPQ2/magi_functional.git) registered for path 'magi_functional'
Submodule 'manga-colorization-v2-custom' (https://github.com/BinhPQ2/manga-colorization-v2-custom.git) registered for path 'manga-colorization-v2-custom'
Cloning into '/kaggle/working/manga_read_along/magi_functional'...
Cloning into '/kaggle/working/manga_read_along/manga-colorization-v2-custom'...
Submodule path 'magi_functional': checked out '56538918e082d03a2a809a623ce470659492334a'
Submodule path 'manga-colorization-v2-custom': checked out 'cace2f95916c684f614554feac04caa6c4780a12'


In [4]:
!pip install -qr /kaggle/working/manga_read_along/magi_functional/requirements_kaggle.txt

Check GPU and set device

In [5]:
import torch
import os

if torch.cuda.is_available():
    !nvidia-smi
    !nvcc --version
else:
    print("GPU is not available")


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on {device}")

Fri Nov  1 21:15:28 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P0             28W /  250W |       3MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Download MAGI model

In [ ]:
from PIL import Image
import numpy as np
from transformers import AutoModel
import torch

model = AutoModel.from_pretrained("ragavsachdeva/magiv2", trust_remote_code=True).to(device).eval()

## Download weight for colorization

In [5]:
!gdown 1qmxUEKADkEM4iYLp1fpPLLKnfZ6tcF-t -O /kaggle/working/manga_read_along/manga-colorization-v2-custom/networks/generator.zip
!mkdir /kaggle/working/manga_read_along/manga-colorization-v2-custom/denoising/models
!gdown 161oyQcYpdkVdw8gKz_MA8RD-Wtg9XDp3 -O /kaggle/working/manga_read_along/manga-colorization-v2-custom/denoising/models/net_rgb.pth

Downloading...
From (original): https://drive.google.com/uc?id=1qmxUEKADkEM4iYLp1fpPLLKnfZ6tcF-t
From (redirected): https://drive.google.com/uc?id=1qmxUEKADkEM4iYLp1fpPLLKnfZ6tcF-t&confirm=t&uuid=5fa29fdd-a87d-4ca5-a991-0584a82351e8
To: /kaggle/working/manga_read_along/manga-colorization-v2-custom/networks/generator.zip
100%|████████████████████████████████████████| 129M/129M [00:01<00:00, 81.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=161oyQcYpdkVdw8gKz_MA8RD-Wtg9XDp3
To: /kaggle/working/manga_read_along/manga-colorization-v2-custom/denoising/models/net_rgb.pth
100%|███████████████████████████████████████| 3.44M/3.44M [00:00<00:00, 189MB/s]


# Magiv2

## Generate json and transcipt file

#### Create raw and character/names list (should apply rename first dumbass, TODO)

In [ ]:
import os
import re
from pathlib import Path

def create_chapter_pages_and_character_bank(manga_folder, character_folder):
    # Create lists for chapter pages and character bank
    chapter_pages = []
    character_bank = {
        "images": [],
        "names": []
    }

#     Iterate through manga images to create chapter_pages
    for image_file in os.listdir(manga_folder):
        if image_file.endswith(('.png', '.jpg', '.jpeg')):  # Check for image file extensions
            # Extract the page number using regex
            match = re.search(r'p(\d+)', image_file)
            if match:
                page_number = int(match.group(1))  # Convert to integer for sorting
                chapter_pages.append((page_number, image_file))  # Store as tuple (page_number, image_file)
            else:
                page_number = image_file.rsplit(".", 1)[0]
                chapter_pages.append((page_number, image_file))  # Store as tuple (page_number, image_file)

    # Sort chapter pages by page number
    chapter_pages.sort(key=lambda x: x[0])
    chapter_pages = [os.path.join(manga_folder, img[1]) for img in chapter_pages]  # Extract just the filenames after sorting

    # Iterate through character images to create character bank
    for char_image_file in os.listdir(character_folder):
        if char_image_file.endswith(('.png', '.jpg', '.jpeg')):  # Check for image file extensions
            # Split the filename to extract character name
            char_name = char_image_file.split('_')[0]  # Get the part before the underscore
            character_bank["images"].append(os.path.join(character_folder, char_image_file))
            character_bank["names"].append(char_name)
    return chapter_pages, character_bank

# Define your folders
# One Piece
# manga_folder = Path("/kaggle/working/magi_functional/data_test/personal_data/One_Piece/raw_manga")
# character_folder = Path("/kaggle/working/magi_functional/data_test/personal_data/One_Piece/character")

# Ruri Dragon
manga_folder = Path("/kaggle/working/manga_read_along/magi_functional/data_test/personal_data/Ruri_Dragon/raw")
character_folder = Path("/kaggle/working/manga_read_along/magi_functional/data_test/personal_data/Ruri_Dragon/character")

# Get chapter pages and character bank
chapter_pages_original, character_bank_original = create_chapter_pages_and_character_bank(manga_folder, character_folder)

chapter_pages_test = chapter_pages_original[:]
character_bank_test = character_bank_original

# Print the results (for debugging)
print("Chapter Pages:")
print(chapter_pages_test)

print("\nCharacter Bank:")
print(character_bank_test)

### Process (OCR → Transcript)

In [ ]:
import os
import json

def read_image(path_to_image):
    with open(path_to_image, "rb") as file:
        image = Image.open(file).convert("L").convert("RGB")
        image = np.array(image)
    return image


chapter_pages = [read_image(x) for x in chapter_pages_test]
character_bank = character_bank_test.copy()
character_bank["images"] = [read_image(x) for x in character_bank_test["images"]]

with torch.no_grad():
    per_page_results = model.do_chapter_wide_prediction(chapter_pages, character_bank, use_tqdm=True, do_ocr=True)

print("Continue with next cell")

### Save transcript and json file

In [ ]:
result_dir = "/kaggle/working/result"
json_output_dir = f"{result_dir}/json_results"
result_image_output_dir = f"{result_dir}/image_results"
transcript_output_dir = f"{result_dir}/transcript.txt"
os.makedirs(json_output_dir, exist_ok=True)  # Create the directory if it doesn't exist  
os.makedirs(result_image_output_dir, exist_ok=True)  # Create the directory if it doesn't exist

transcript = []
for i, (image, page_result) in enumerate(zip(chapter_pages, per_page_results)):
    image_name_ext = os.path.basename(chapter_pages_test[i]) 
    # Split the image name and its extension
    image_name, image_extension = os.path.splitext(image_name_ext)
    
#     model.visualise_single_image_prediction(image, page_result, os.path.join(result_image_output_dir, f"{image_name}.png")) # enable this if you want to see the result image included with all the annotation box
    # Save page_result to JSON
    json_file_path = os.path.join(json_output_dir, f"{image_name}.json")
    with open(json_file_path, 'w') as json_file:
        json.dump(page_result, json_file, indent=4)

    speaker_name = {
        text_idx: page_result["character_names"][char_idx] for text_idx, char_idx in page_result["text_character_associations"]
    }
    
    transcript.append(f"<page>{image_name}<endpage>")
    for j in range(len(page_result["ocr"])):
        if not page_result["is_essential_text"][j]:
            continue
        name = speaker_name.get(j, "unsure") 
        transcript.append(f"<name>{name}<endname>: {page_result['ocr'][j]}")
with open(transcript_output_dir, "w") as fh:
    for line in transcript:
        fh.write(line + "\n")

print("\n\nDone you WEEEEB!")

## Generate video from Magi json file

In [29]:
!python /kaggle/working/manga_read_along/magi_functional/main.py

Deleted images
Processed and saved modified full page images for: 0000.jpg using json file: magi_functional/data_test/personal_data/Ruri_Dragon/json_results/renamed/0000.json
Processed and saved modified full page images for: 0001.jpg using json file: magi_functional/data_test/personal_data/Ruri_Dragon/json_results/renamed/0001.json
Processed and saved modified full page images for: 0002.jpg using json file: magi_functional/data_test/personal_data/Ruri_Dragon/json_results/renamed/0002.json
Processed and saved modified full page images for: 0003.jpg using json file: magi_functional/data_test/personal_data/Ruri_Dragon/json_results/renamed/0003.json
Processed and saved modified full page images for: 0004.jpg using json file: magi_functional/data_test/personal_data/Ruri_Dragon/json_results/renamed/0004.json
Processed and saved modified full page images for: 0005.jpg using json file: magi_functional/data_test/personal_data/Ruri_Dragon/json_results/renamed/0005.json
Processed and saved modif

## Download image result

In [33]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -rj {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

download_file("image_results", "/kaggle/working/manga_read_along/test_output_final")

/kaggle/working/image_results.zip

# Colorization

In THEORY, this should work (haven't tested yet)

In [ ]:
!python "/kaggle/working/manga_read_along/manga-colorization-v2-custom/main.py"

or run the below code

In [ ]:
!python "/kaggle/working/manga_read_along/manga-colorization-v2-custom/inference_v2.py" -p "/kaggle/working/manga_read_along/magi_functional/data_test/personal_data/Ruri_Dragon/raw/" -des_path "/kaggle/working/manga_read_along/manga-colorization-v2-custom/denoising/models/net_rgb.pth" -gen "/kaggle/working/manga_read_along/manga-colorization-v2-custom/networks/generator.zip" -s "./test_output_coloring" -ds 0 --gpu

# Text to Speech

## Install Library

In [11]:
!pip install -q TTS
!pip install -q pydub

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
ucxx 0.39.1 requires libucx>=1.15.0, which is not installed.
albucore 0.0.17 requires numpy>=1.24, but you have numpy 1.22.0 which is incompatible.
albumentations 1.4.17 requires numpy>=1.24.4, but you have numpy 1.22.0 which is incompatible.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you h

In [13]:
import librosa
import librosa
import numpy as np
import soundfile as sf
from IPython.display import Audio
from TTS.api import TTS
from unittest.mock import patch
import re

In [14]:
# Mock input to automatically respond with 'y'
with patch('builtins.input', return_value='y'):
    tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|█████████▉| 1.86G/1.87G [00:43<00:00, 42.9MiB/s]
100%|██████████| 1.87G/1.87G [00:44<00:00, 42.2MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 23.3kiB/s]

100%|██████████| 361k/361k [00:00<00:00, 1.59MiB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 130iB/s]
 50%|█████     | 3.90M/7.75M [00:00<00:00, 39.0MiB/s]

 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


/opt/conda/lib/python3.10/site-packages/TTS/tts/layers/xtts/xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.speakers = torch.load(speaker_file_path)
10

Remember to add RAVDESS Emotional speech audio (Female datasets to kaggle input, yes it's stupid af but its a bandage solution for now バカ)

## Code

In [ ]:
voice_mapping = {
'other': "/kaggle/input/ravdess-emotional-speech-audio/Actor_02/03-01-01-01-01-01-02.wav",
'ruri': "/kaggle/input/ravdess-emotional-speech-audio/Actor_04/03-01-01-01-01-01-04.wav",
'teacher': "/kaggle/input/ravdess-emotional-speech-audio/Actor_01/03-01-01-01-01-01-01.wav",
'mom': "/kaggle/input/ravdess-emotional-speech-audio/Actor_08/03-01-01-01-01-01-08.wav",
'unsure': "/kaggle/input/ravdess-emotional-speech-audio/Actor_10/03-01-01-01-01-01-10.wav",
'ukka': "/kaggle/input/ravdess-emotional-speech-audio/Actor_12/03-01-01-01-01-01-12.wav"
}
 

In [15]:
from magi_functional.utils.utils import rename_image_to_correct_format, get_digit_number_for_name_format, generate_name_format

images_folder = '/kaggle/working/manga_read_along/test_output_coloring'

number_of_digit_for_name = get_digit_number_for_name_format(images_folder)
name_format = generate_name_format(number_of_digit_for_name)
print(name_format)

page_{:04}_panel_{:04}_bubble_{:04}{}


In [57]:
import os
import random
import re
from pydub import AudioSegment
import shutil

# Function to parse the transcript from a text file
def parse_transcript(transcript_path, line_end: int = None):
    pages = []
    current_page = None

    with open(transcript_path, 'r') as file:
        content = file.readlines()
        
    lines_to_process = content[:line_end] if line_end is not None else content
    
    for line in lines_to_process:
        if line.startswith("<page>"):
            # Start a new page
            if current_page is not None:
                pages.append(current_page)
            current_page = {"page": line[6:8], "lines": []}  # Extract page number 
        elif line.startswith("<name>") and current_page is not None:
            # Extract character name and dialogue
            match = re.match(r"<name>([^<]+)<endname>:\s*(.+)", line)
            if match:
                character = match.group(1).lower()  # Lowercase for consistency
                dialogue = match.group(2)
                current_page["lines"].append((character, dialogue))

    # Add the last page if it exists
    if current_page is not None:
        pages.append(current_page)
        
    return pages

# Function to get all voice files from the specified directory
def get_voice_files(directory):
    all_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".wav"):
                all_files.append(os.path.join(root, file))
    return all_files

# Function to filter voice files into male and female categories
def filter_voice_files(files):
    male_files = []
    female_files = []
    
    for file in files:
        # Extract the actor number from the filename
        match = re.search(r"(\d{2})\.wav$", file)
        if match:
            actor_number = int(match.group(1))
            if actor_number % 2 == 0:  # Even numbers are female
                female_files.append(file)
            else:  # Odd numbers are male
                male_files.append(file)
                
    return male_files, female_files

# Function to randomly select voice files for characters
def select_voice_files_for_characters(characters):
    all_files = get_voice_files(voice_bank)
    male_files, female_files = filter_voice_files(all_files)

    selected_files = {}
    used_files = set()  # To keep track of used voice files

    for character in characters:
        if character in male_characters:
            # Select a random male voice that hasn't been used
            available_male_files = list(set(male_files) - used_files)
            if available_male_files:
                selected_files[character] = random.choice(available_male_files)
                used_files.add(selected_files[character])
        else:
            # Select a random female voice that hasn't been used, if available
            available_female_files = list(set(female_files) - used_files)
            if available_female_files:
                selected_files[character] = random.choice(available_female_files)
                used_files.add(selected_files[character])
            else:
                # Select any remaining file for other characters, ensuring it's not already used
                available_files = list(set(all_files) - used_files)
                if available_files:
                    selected_files[character] = random.choice(available_files)
                    used_files.add(selected_files[character])
    
    return selected_files

# Function to convert text to speech for a character
def voice_character(character, text, page_number, bubble_number, selected_files, save_directory):
    speaker_wav = selected_files.get(character)

    if speaker_wav:
        audio_output_filename = name_format.format(int(page_number), 0, bubble_number+1, ".wav")
        output_filename = os.path.join(save_directory, audio_output_filename)
        output = tts.tts_to_file(text=text, speaker_wav=speaker_wav, language="en")
        os.rename("output.wav", output_filename)  # Rename the default output file
        return output_filename
    else:
        raise ValueError(f"Character '{character}' not found in voice mapping.")

# Function to process the transcript and create audio files
def text2speech(pages, selected_files, save_directory):
    output_files = []
    time_file = []

    for page in pages:
        page_number = page["page"]
        for bubble_number, (character, dialogue) in enumerate(page["lines"]):
            try:
                output = voice_character(character, dialogue, page_number, bubble_number, selected_files, save_directory)
                output_files.append(output)
            except ValueError as e:
                print(e)

    return output_files

def duration_calculation():
    audio = AudioSegment.from_file(output_filename)
    duration_in_seconds = len(audio) / 1000
    return duration_in_seconds

In [61]:
# Define the directory where the RAVDESS files are located
voice_bank = "/kaggle/input/ravdess-emotional-speech-audio/"
# Define male characters for the check
male_characters = ['teacher']
transcript_path = "/kaggle/working/manga_read_along/magi_functional/data_test/personal_data/Ruri_Dragon/transcript.txt"

save_directory = "/kaggle/working/audio_output/"

if os.path.exists(save_directory):
    shutil.rmtree(save_directory)
os.makedirs(save_directory, exist_ok=True)

# Get unique characters from the transcript
pages = parse_transcript(transcript_path, line_end = 20) # delete line_end to get audio of all page
characters = {line[0] for page in pages for line in page["lines"]}  # Get unique characters

# Select voice files for characters
selected_files = select_voice_files_for_characters(characters)

# # Call the function to process the transcript and generate audio files
output_files = text2speech(pages, selected_files, save_directory)

 > Text splitted to sentences.
['Eh...?']
 > Processing time: 0.5989968776702881
 > Real-time factor: 0.3508255724774185
 > Text splitted to sentences.
['All of a sudden']
 > Processing time: 0.4944484233856201
 > Real-time factor: 0.3274443697637231
 > Text splitted to sentences.
['I grew horns']
 > Processing time: 0.5917227268218994
 > Real-time factor: 0.3265790480181939
 > Text splitted to sentences.
['Am I really human?']
 > Processing time: 0.5918266773223877
 > Real-time factor: 0.3184736976512751
 > Text splitted to sentences.
['Mom, about this...']
 > Processing time: 0.701024055480957
 > Real-time factor: 0.32278609303698425
 > Text splitted to sentences.
['Oh, Mor- Ning Honey.']
 > Processing time: 1.2274980545043945
 > Real-time factor: 0.3432026285988778
 > Text splitted to sentences.
['This thing on my head ...']
 > Processing time: 1.3988063335418701
 > Real-time factor: 0.3511987572257952
 > Text splitted to sentences.
["It's horns."]
 > Processing time: 0.520362138748

# Download contents

## Download separate files

### Download json

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -rj {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

download_file("json_results", f"{json_output_dir}/*.json")

### Download image results

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -rj {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

download_file("image_results", result_image_output_dir)

### Download Audio

In [64]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -rj {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

download_file("audio_results", "/kaggle/working/audio_output")

/kaggle/working/audio_results.zip

In [63]:
!rm -rf /kaggle/working/audio_results.zip

### Download transcript

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -rj {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))
download_file("transcript", "/kaggle/working/result/transcript.txt")

## Download all

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(download_file_name, source_path):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip -r {zip_name} {source_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))
    

!cp -r result_dir /
download_file("result", result_dir)

## Edit kaggle

In [22]:
process_file = '''
import json
import math
import os
from PIL import Image, ImageDraw
from utils.utils import sort_files

def get_digit_number_for_name_format(directory_path, buffer_number: int = 2):
    image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.tif', '.webp', '.heif', '.ico')
    num_images = len([name for name in os.listdir(directory_path) if name.lower().endswith(image_extensions)])
     
    if num_images > 0:
        number_of_digit_for_name = math.ceil(math.log10(num_images+1)) + buffer_number
    else: 
        number_of_digit_for_name = buffer_number
    return number_of_digit_for_name

def generate_name_format(number_of_digit_for_name: int):
    return f"page_{{:0{number_of_digit_for_name}}}_panel_{{:0{number_of_digit_for_name}}}_bubble_{{:0{number_of_digit_for_name}}}{{}}"

def read_coordinates(json_file_path: str):
    """Read essential text coordinates from the given JSON file."""
    with open(json_file_path, 'r') as f:
        data = json.load(f)
    return [
        text for text, is_essential in zip(data["texts"], data["is_essential_text"]) if is_essential
    ], data["panels"]  # Return text coordinates and panel coordinates

def process_full_page(images_dir: str, json_file_path: str, save_path: str = "./panel_images", name_format: str = "page_{:03}_panel_{:03}_bubble_{:03}{}"):
    os.makedirs(save_path, exist_ok=True)

    # Extract the text coordinates
    text_coords, _ = read_coordinates(json_file_path)
    
    total_length_text = len(text_coords)
    image_name_ext = os.path.basename(images_dir)

    # Split the image name and its extension
    image_name, image_extension = os.path.splitext(image_name_ext)
    image_name = int(image_name)

    with Image.open(images_dir) as img:
        # Create a draw object on the original image copy
        draw = ImageDraw.Draw(img)

        # Initialize the modified panel image path
        modified_panel_image_path = os.path.join(save_path, name_format)
        text_order = total_length_text

        # Save the modified panel image with the last computed bubble index
        img.save(modified_panel_image_path.format(image_name, 0, text_order, image_extension))

        # Loop through each set of coordinates in reverse order
        for box_index, box in enumerate(reversed(text_coords)):
            x1, y1, x2, y2 = map(int, box)  # Ensures all values are integers

            # Initialize max color and max value
            max_color = (0, 0, 0)  # Start with black
            max_value = 0

            # Iterate through the box's pixels
            for y in range(y1, y2):
                for x in range(x1, x2):
                    # Get the pixel color using absolute coordinates
                    color = img.getpixel((x, y))
                    color_value = sum(color[:3])  # Sum RGB values

                    # Update max color if the current pixel's value is higher
                    if color_value > max_value:
                        max_value = color_value
                        max_color = color

            # Fill the rectangle with the max color
            draw.rectangle(box, fill=max_color)

            # Save modified image path for the current chat bubble
            text_order = total_length_text - (box_index + 1)
            modified_image_path = modified_panel_image_path.format(image_name, 0, text_order, image_extension)

            # Instead of saving here, save it once at the end
            img.save(modified_image_path, format='JPEG')

    print(f"Processed and saved modified full page images for: {image_name_ext} using json file: {json_file_path}")

def process_panel_view(images_dir: str, json_file_path: str, save_path: str = "./panel_images", name_format: str = "page_{:03}_panel_{:03}_bubble_{:03}{}"):
    os.makedirs(save_path, exist_ok=True)

    text_coords, panel_coords = read_coordinates(json_file_path)
    total_length_text = len(text_coords)
    image_name_ext = os.path.basename(images_dir)

    # Split the image name and its extension
    image_name, image_extension = os.path.splitext(image_name_ext)
    image_name = int(image_name)

    with Image.open(images_dir) as img:
        # Create a copy of the image for drawing
        original_img = img.copy()

        # Loop through each panel
        for panel_index, panel in enumerate(panel_coords):
            # Create a panel-specific image by cropping the original image
            panel_box = (panel[0], panel[1], panel[2], panel[3])  # box format: (x1, y1, x2, y2)
            panel_image = original_img.crop(panel_box)

            # Create a draw object on the panel image copy
            draw = ImageDraw.Draw(panel_image)

            # Initialize the modified panel image path
            modified_panel_image_path = os.path.join(save_path, name_format)
            text_order = total_length_text

            # Save the modified panel image with the last computed bubble index
            panel_image.save(modified_panel_image_path.format(image_name, panel_index, text_order, '.jpg'))
            
            # Loop through each set of coordinates in reverse order
            for box_index, box in enumerate(reversed(text_coords)):
                # Calculate the center point of the chatbox
                x_center = (box[0] + box[2]) // 2
                y_center = (box[1] + box[3]) // 2

                # Check if the center point is inside the panel
                if panel[0] <= x_center <= panel[2] and panel[1] <= y_center <= panel[3]:
                    # Calculate the position of the box relative to the panel
                    relative_box = (
                        box[0] - panel[0],
                        box[1] - panel[1],
                        box[2] - panel[0],
                        box[3] - panel[1]
                    )

                    # Get the coordinates of the relative box and convert to integers
                    x1, y1, x2, y2 = map(int, relative_box)  # Ensures all values are integers

                    # Initialize max color and max value
                    max_color = (0, 0, 0)  # Start with black
                    max_value = 0

                    # Iterate through the box's pixels
                    for y in range(y1, y2):
                        for x in range(x1, x2):
                            # Get the pixel color using absolute coordinates
                            color = original_img.getpixel((x + panel[0], y + panel[1]))
                            color_value = sum(color[:3])  # Sum RGB values

                            # Update max color if the current pixel's value is higher
                            if color_value > max_value:
                                max_value = color_value
                                max_color = color

                    # Fill the rectangle with the max color
                    draw.rectangle(relative_box, fill=max_color)

                    # Save modified image path for the current chat bubble
                    text_order = total_length_text - (box_index + 1)
                    modified_image_path = modified_panel_image_path.format(image_name, panel_index, text_order, '.jpg')

                    # Instead of saving here, save it once at the end
                    panel_image.save(modified_image_path, format='JPEG')

    print(f"Processed and saved modified panel view images for: {image_name_ext}")

def process_all_images_and_jsons(images_folder: str, json_folder: str, save_path: str = "./panel_images", name_format: str = "page_{:03}_panel_{:03}_bubble_{:03}{}", nuke: bool = False, panel_view: bool = False):
    
    image_extensions = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff'}
    if nuke:
        try:
            for filename in os.listdir(save_path):
                # Check if the file has an image extension
                if os.path.splitext(filename)[1].lower() in image_extensions:
                    file_path = os.path.join(save_path, filename)
                    os.remove(file_path)
            print(f"Deleted images")
        except FileNotFoundError as e:
            print(f"Error: {e}")

    # Sort files using the existing sort_files function
    sorted_image_files = sort_files(images_folder)
    sorted_json_files = sort_files(json_folder)
    
    # Get all image and json file names
    image_files = [f for f in sorted_image_files if os.path.splitext(f)[1].lower() in image_extensions]
    json_files = [f for f in sorted_json_files if f.lower().endswith('.json')]

    if len(image_files) != len(json_files):
        print(f"Number of images and json files do not match! Length of images: {len(image_files)}, Length of jsons: {len(json_files)}")
    else:
        for image_file in image_files:
            # Construct full image path
            image_path = os.path.join(images_folder, image_file)

            # Corresponding json file name (assuming they have the same base name)
            json_file_name = os.path.splitext(image_file)[0] + '.json'
            if json_file_name in json_files:
                json_path = os.path.join(json_folder, json_file_name)

                # Process the image and json based on panel_view flag
                if panel_view:
                    process_panel_view(image_path, json_path, save_path, name_format)
                else:
                    process_full_page(image_path, json_path, save_path, name_format)
            else:
                print(f"JSON file not found for image: {image_file}")


if __name__ == "__main__":
    images_folder = 'magi_functional/data_test/personal_data/Ruri_Dragon/raw'
    json_folder = 'magi_functional/data_test/personal_data/Ruri_Dragon/json_results'
    save_path = "magi_functional/data_test/code/test_lab/panel_images_full_chapter"
    number_of_digit_for_name = get_digit_number_for_name_format(directory_path = images_folder, buffer_number = 2)
    name_format =generate_name_format(number_of_digit_for_name)

    nuke_option = True
    panel_view_option= False

    process_all_images_and_jsons(images_folder = images_folder, json_folder = json_folder, save_path = save_path, name_format = name_format, nuke = nuke_option, panel_view = panel_view_option)

'''

with open("/kaggle/working/manga_read_along/magi_functional/utils/process_raw_from_json.py", "w") as file:
    file.write(process_file)
    print("Done")

Done


In [23]:
main_file = '''
import os
from utils.img2mp4 import create_video_from_images
from utils.process_raw_from_json import process_all_images_and_jsons, get_digit_number_for_name_format, generate_name_format
from utils.utils import rename_image_to_correct_format

def main(process_image=True, create_video=True):
    images_folder = 'test_output_coloring'
    json_folder = 'magi_functional/data_test/personal_data/Ruri_Dragon/json_results'

    save_path = "test_output_final"
    number_of_digit_for_name = get_digit_number_for_name_format(images_folder)
    name_format = generate_name_format(number_of_digit_for_name)

    formated_image_path = os.path.join(images_folder, 'renamed')
    formated_images_folder = rename_image_to_correct_format(images_folder, formated_image_path, num_digits=number_of_digit_for_name)
    formated_json_path = os.path.join(json_folder, 'renamed')
    formated_json_folder = rename_image_to_correct_format(json_folder, formated_json_path, num_digits=number_of_digit_for_name)

    nuke_option = True
    panel_view_option = False

    if process_image:
        process_all_images_and_jsons(images_folder = formated_images_folder, json_folder = formated_json_folder, save_path = save_path, name_format = name_format, nuke = nuke_option, panel_view = panel_view_option)

    if create_video:
        create_video_from_images(image_dir=save_path, output_video_path=save_path, name_format=name_format, use_padding=True)

if __name__ == "__main__":
    main(process_image=True, create_video=True)
'''

with open("/kaggle/working/manga_read_along/magi_functional/main.py", "w") as file:
    file.write(main_file)
    print("Done")

Done


In [93]:
utils = '''
import math
import os
import re
import shutil


def get_digit_number_for_name_format(directory_path, buffer_number: int = 2):
    image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.tif', '.webp', '.heif', '.ico')
    num_images = len([name for name in os.listdir(directory_path) if name.lower().endswith(image_extensions)])
     
    if num_images > 0:
        number_of_digit_for_name = math.ceil(math.log10(num_images+1)) + buffer_number
    else: 
        number_of_digit_for_name = buffer_number
    return number_of_digit_for_name

def fk_you(number_of_digit_for_name: int):
    name_format = f"page_{{:0{number_of_digit_for_name}}}_panel_{{:0{number_of_digit_for_name}}}_bubble_{{:0{number_of_digit_for_name}}}{{}}"
    return name_format

def rename_image_to_correct_format(images_folder, new_folder, num_digits: int =3):
    if os.path.exists(new_folder):
        shutil.rmtree(new_folder)

    os.makedirs(new_folder, exist_ok=True)

    sorted_images = sort_files(images_folder)

    for idx, image_path in enumerate(sorted_images):
        original_path = os.path.join(images_folder, image_path)
        if os.path.isfile(original_path):  # Ensure it's a file
            original_ext = os.path.splitext(image_path)[1]
            new_name = f"{idx:0{num_digits}}{original_ext}"
            new_path = os.path.join(new_folder, new_name)
            shutil.copy2(original_path, new_path)
    return new_folder

def sort_files(directory):
    files = []
    
    for file in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, file)):
            files.append(file)

    sorted_list = sorted(files, key=lambda filename: [int(part) if part.isdigit() else part for part in re.split(r'(\d+)', filename)])
    return sorted_list
'''

with open("/kaggle/working/manga_read_along/utils.py", "w") as file:
    file.write(utils)
    print("Done")

Done
